# Init
Get the data from the EventBrite API and process it into a list of events.
Cache it to an `events.json` file stored in the file system.

In [ ]:
import json
import pandas as pd
from datetime import datetime
from pathlib import Path
from requests import get
import matplotlib.pyplot as plt

plt.close("all")

pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

if not Path("events.json").is_file():


    # documentation: https://www.eventbrite.nl/platform/api
    # new API key from: https://www.eventbrite.nl/platform/api-keys/
    #   note: each visit of this page creates a new API key at https://www.eventbrite.nl/account-settings/apps
    # API key stored in Bitwarden vault Bas
    auth_header = {"Authorization": "Bearer <api-key>"}

    events = []
    has_more_items = True
    page = 0

    while has_more_items:
        page += 1
        r = get(f"https://www.eventbriteapi.com/v3/organizations/187233351803/events/?order_by=created_asc&page={page}&expand=venue", headers=auth_header)
        events_json = r.json()
        for event_json in events_json["events"]:
            event_json["event_date"] = event_json["start"]["local"]
            ra = get(f"https://www.eventbriteapi.com/v3/events/{event_json['id']}/attendees", headers=auth_header)
            event_json['attendees'] = ra.json()
            rd = get(f"https://www.eventbriteapi.com/v3/events/{event_json['id']}/description", headers=auth_header)
            event_json['description_ext'] = rd.json()
            events.append(event_json)
        has_more_items = events_json["pagination"]["has_more_items"]

    with open("events.json", 'w') as events_file:
        events_file.write(json.dumps(events))
else:
    with open("events.json", 'r') as events_file:
        events = json.load(events_file)


Prepare some functions and filter out events that are not for ninjas.

In [ ]:

def get_sold_tickets(event):
    return event['attendees']['pagination']['object_count']


def get_checkins(event):
    return len([attendee for attendee in event['attendees']['attendees'] if attendee['checked_in']])

def as_datetime(event):
    return datetime.strptime(event["event_date"], "%Y-%m-%dT%H:%M:%S")

def as_month(event):
    return datetime.strftime(as_datetime(event), '%m')

def as_date(event):
    return datetime.strftime(as_datetime(event), '%Y-%m-%d')

known_locations = [
    "zwanenveld",
    "biezantijn",
    "mariënburg",
    "online",
    "valkhof"
]

def location(event: dict) -> str:
    loc = 'online' if event['online_event'] else event['venue']['name']
    for known_location in known_locations:
        if known_location in loc.lower():
            return known_location
    return loc

ids_to_remove = [
37396107765,
43018135401,
49624002728,
]
for event in list(events):
    if int(event['id']) in ids_to_remove:
        events.remove(event)


# Overview of all dojos over time in a table

In [ ]:
dojos = []
for event in events:
    dojos.append([
        as_month(event), 
        as_date(event), 
        'online' if event['online_event'] else event['venue']['name'], 
        event['name']['text'], 
        event['capacity'], 
        get_sold_tickets(event), 
        get_checkins(event)
    ])

pd.DataFrame(dojos, columns=["maand", "datum", "locatie", "onderwerp", "aantal tickets", "gekocht", "ingecheckt"])

In [ ]:
checkins_over_time = {
    'datum': [as_date(event) for event in events],
    'checkins': [get_checkins(event) if not event['online_event'] else 0 for event in events],
    'online checkins': [get_checkins(event) if event['online_event'] else 0  for event in events],
}
df = pd.DataFrame(checkins_over_time)
df.plot(x='datum', y=['checkins', 'online checkins'], kind='bar', stacked=True, width=0.8, figsize=(20,8), colormap='jet', title='checkins over tijd')


In [ ]:

locations = list(set([location(event) for event in events]))

checkins = {
    'datum': [as_date(event) for event in events],
}
for loc in locations:
    checkins[loc] = [get_checkins(event) if location(event) == loc else 0 for event in events]

df = pd.DataFrame(checkins)
df.plot(x='datum', y=locations, kind='bar', figsize=(20,8), stacked=True, width=0.8, colormap='hsv', title='checkins over tijd per locatie')
    


In [ ]:
# per maand
months = {}
for event in events:
    if not event['online_event']:
        month = as_month(event)
        if not month in months:
            months[month] = {}
        months[month][as_date(event)] = get_checkins(event)

monthly = []
print("maand,min,avg,max")
for month in sorted(months):
    total = 0
    min = 100
    max = 0
    for datum in months[month]:
        checkins = months[month][datum]
        total += checkins
        if checkins < min:
            min = checkins
        if checkins > max:
            max = checkins
        # print(f"{month},{datum},{checkins}")
    avg = total/len(months[month])
    monthly.append([month, min, avg, max])
    # print(f"{month},{min},{avg - min},{max - avg},{avg},{max}")

df = pd.DataFrame(monthly, columns=['maand', 'min', 'gemiddeld', 'max'])
df.plot(x='maand', y=['min', 'gemiddeld', 'max'], kind='bar', title='checkins per maand van het jaar')
